<a href="https://colab.research.google.com/github/sayanbanerjee32/ms-phi2-qlora-oasst1/blob/main/ms_phi2_custom_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq accelerate transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130

In [2]:
# assignment repo
!git clone https://github.com/sayanbanerjee32/ms-phi2-qlora-oasst1.git

Cloning into 'ms-phi2-qlora-oasst1'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), done.
Resolving deltas: 100% (1/1), done.


In [3]:
## move python files to outside
!mv ms-phi2-qlora-oasst1/app.py .

In [5]:
from huggingface_hub import HfApi
from google.colab import userdata
userdata.get('HF_TOKEN')
api = HfApi()

In [5]:
# !pip install datasets

In [6]:
# from datasets import load_dataset
# import pandas as pd
# valid_dataset = load_dataset("OpenAssistant/oasst1", split="validation")
# valid_df = valid_dataset.to_pandas()
# valid_df.loc[valid_df.lang == 'en'].head(50)

In [7]:
# def add_tree_level(df):
#     """helper function to add tree level to a df"""

#     # if tree level already exists, return df
#     if "tree_level" in df.columns:
#         return df

#     else:
#         tree_level_map = {}

#         # iterate over rows in df
#         for i, row in df.iterrows():
#             message_id = row["message_id"]
#             parent_id = row["parent_id"]

#             # if parent_id is None, then it is a root message
#             if parent_id is None:
#                 tree_level_map[message_id] = 0
#             # if parent_id is the same as message_tree_id, then it is a direct reply to the root message
#             elif parent_id == row["message_tree_id"]:
#                 tree_level_map[message_id] = 1
#             # else just look up the tree level of the parent_id and add 1
#             else:
#                 tree_level_map[message_id] = tree_level_map[parent_id] + 1

#         # create a df from the tree_level_map and merge it with the original df
#         df_tree_level_map = (
#             pd.DataFrame.from_dict(tree_level_map, orient="index", columns=["tree_level"])
#             .reset_index()
#             .rename(columns={"index": "message_id"})
#         )

#         return df.merge(df_tree_level_map, on="message_id")


# # convert to function
# def perproc_data(df, message_tree_id):
#     # look at all data for this message tree
#     df_message_tree = df.query(f"message_tree_id == '{message_tree_id}'").sort_values("created_date")
#     # add tree level to df
#     df_message_tree = add_tree_level(df_message_tree)
#     df_message_tree = df_message_tree.sort_values(['tree_level','rank'])

#     text = ""
#     # root message
#     row = df_message_tree.loc[df_message_tree.tree_level == 0]
#     text = "<|" + row["role"].values[0] + "|>"+ row["text"].values[0] + "<|endoftext|>"
#     text = text.replace("\n", " ")
#     while True:
#         children = df_message_tree[df_message_tree.parent_id == row.message_id.values[0]]
#         if len(children) == 0:
#             break
#         elif len(children) > 1:
#             row = children.loc[((children['role']== 'prompter') | (children['rank'] == 0.0))]
#         else:
#             row = children

#         if len(row) == 0: break

#         text += "<|" + row["role"].values[0] + "|>"+ row["text"].values[0] + "<|endoftext|>"
#         text = text.replace("\n", " ")
#         # print(text)
#     return text

# perproc_data(valid_df, '75e05b0b-c904-4f8a-a617-b561e78e6f1d')

## Load a fine-tuned model from Drive and run inference

In [8]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from transformers import pipeline
# import gradio as gr
# import time

# REPO_ID = "sayanbanerjee32/ms-phi2-qlora-oasst1"

# model = AutoModelForCausalLM.from_pretrained(REPO_ID)
# tokenizer = AutoTokenizer.from_pretrained(REPO_ID)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [15]:
# def generate_text(prompt, chat_history, num_new_tokens = 100):
#     # prompt = "<|prompter|>What is 2 + 2?<|endoftext|><|assistant|>"  # change to your desired prompt
#     # change to the number of new tokens you want to generate
#     # Count the number of tokens in the prompt
#     input_prompt = ''
#     if len(chat_history) > 0:
#         input_prompt += "<|prompter|>" + chat_history[-1][0] + "<|endoftext|><|assistant|>" + chat_history[-1][1] + "<|endoftext|>"
#     input_prompt += "<|prompter|>" + prompt + "<|endoftext|><|assistant|>"
#     print(input_prompt)
#     num_prompt_tokens = len(tokenizer(input_prompt)['input_ids'])
#     print(num_prompt_tokens)
#     # Calculate the maximum length for the generation
#     max_length = num_prompt_tokens + num_new_tokens
#     gen = pipeline('text-generation', model=model,
#                 tokenizer=tokenizer, max_length= max_length )
#     result = gen(prompt)
#     return result[0]['generated_text'].replace(prompt, '')

# with gr.Blocks() as demo:
#     gr.HTML("<h1 align = 'center'> Chat </h1>")
#     gr.HTML("<h4 align = 'center'> ChatBot powered by Microsoft-Phi-2 finetuned on OpenAssistant dataset</h4>")

#     chatbot = gr.Chatbot()
#     msg = gr.Textbox(label = "Ask your question here")
#     gr.Examples(["What do you think about ChatGPT?",
#                 "How would the Future of AI in 10 Years look?",
#                 # "Write a announcement tweet for medium.com readers about the new blogpost on 'Open Assistant is open source ChatGPT that you don\'t wanna miss out'",
#                 "Please implement the Timsort algorithm on Lean 4 and explain your code",
#                 "How do I build a PC?"],
#                 inputs = msg)
#     clear = gr.ClearButton([msg, chatbot])

#     def respond(message, chat_history):
#         # print(message)
#         # print(chat_history)
#         bot_message = generate_text(message, chat_history)
#         chat_history.append((message, bot_message))
#         time.sleep(2)
#         return "", chat_history

#     msg.submit(respond, [msg, chatbot], [msg, chatbot])

# # for collab
# demo.launch(debug=True)

# # if __name__ == '__main__':
# #     demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7437f6d91e1d521534.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<|prompter|>What do you think about ChatGPT?<|endoftext|><|assistant|>
22
<|prompter|>What do you think about ChatGPT?<|endoftext|><|assistant|>Do you think it will replace human teachers in the future? Why or why not?’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’’<|endoftext|><|prompter|>I would like to ask you<|endoftext|><|assistant|>
155
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7437f6d91e1d521534.gradio.live


In [7]:
import app

### Upload to huggingface spaces

In [2]:
import os
os.makedirs('to_upload', exist_ok=True)

In [3]:
!cp app.py to_upload
!cp ms-phi2-qlora-oasst1/requirements.txt to_upload

In [8]:
api.upload_folder(
    folder_path="./to_upload",
    repo_id="sayanbanerjee32/askme-ms-phi2-qlora-oasst1",
    repo_type="space",
)


CommitInfo(commit_url='https://huggingface.co/spaces/sayanbanerjee32/askme-ms-phi2-qlora-oasst1/commit/df4f6e77ed4253288948d6f2ffcd615d37b44209', commit_message='Upload folder using huggingface_hub', commit_description='', oid='df4f6e77ed4253288948d6f2ffcd615d37b44209', pr_url=None, pr_revision=None, pr_num=None)